In [106]:
import numpy as np
import librosa
import os
import pandas as pd

In [107]:
# MFCCs

def compute_mfccs(signal, sample_rate, n_mfcc=13):
    """
    Tính toán Mel-frequency cepstral coefficients (MFCCs) của tín hiệu âm thanh.

    Parameters:
        signal (numpy array): Tín hiệu âm thanh.
        sample_rate (int): Tần số lấy mẫu của tín hiệu âm thanh.
        n_mfcc (int): Số lượng MFCCs.

    Returns:
        numpy array: Trung bình của MFCCs.
    """
    mfccs = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=n_mfcc)
    return np.mean(mfccs.T, axis=0)

In [108]:
# def compute_chroma(signal, sample_rate):
#     """
#     Tính toán Chroma của tín hiệu âm thanh.

#     Parameters:
#         signal (numpy array): Tín hiệu âm thanh.
#         sample_rate (int): Tần số lấy mẫu của tín hiệu âm thanh.

#     Returns:
#         numpy array: Trung bình của Chroma.
#     """
#     chroma = librosa.feature.chroma_stft(y=signal, sr=sample_rate)
#     return chroma.mean(axis=1)

In [109]:
def compute_zcr(signal):
    """
    Tính toán Zero Crossing Rate (ZCR) của tín hiệu âm thanh.

    Parameters:
        signal (numpy array): Tín hiệu âm thanh.

    Returns:
        float: ZCR của tín hiệu âm thanh.
    """
    return np.mean(np.abs(np.diff(np.sign(signal))))

In [110]:
def compute_spectral_flux(spectrogram):
    """
    Tính toán Spectral Flux của phổ âm thanh.

    Parameters:
        spectrogram (numpy array): Phổ âm thanh.

    Returns:
        float: Spectral Flux của phổ âm thanh.
    """
    return np.mean(np.abs(np.diff(spectrogram, axis=1)))

In [111]:
def compute_spectral_centroid(spectrogram, frequencies):
    """
    Tính toán Spectral Centroid của phổ âm thanh.

    Parameters:
        spectrogram (numpy array): Phổ âm thanh.
        frequencies (numpy array): Tần số của phổ âm thanh.

    Returns:
        float: Spectral Centroid của phổ âm thanh.
    """
    return np.sum(frequencies * spectrogram) / np.sum(spectrogram)

In [112]:
def extract_pitch(signal, sample_rate):
    # Tính toán pitch sử dụng hàm estimate_tuning của Librosa
    pitch, _ = librosa.core.piptrack(y=signal, sr=sample_rate)
    
    # Lấy giá trị pitch trung bình
    mean_pitch = pitch.mean()
    
    return mean_pitch

In [113]:
def feature_engineering_for_file(audio_file, n_mfcc):
    signal, sample_rate = librosa.load(audio_file, sr=None)
    if len(signal) >= 2048:
      return compute_mfccs(signal, sample_rate, n_mfcc), compute_zcr(signal), extract_pitch(signal, sample_rate)
    return None

In [114]:
def process_file(label, input_folder_path, n_mfcc):
    """
    Xử lý tất cả các tệp WAV trong thư mục để tìm các phân đoạn nói.
    """
    # Tạo một DataFrame để lưu trữ đặc trưng MFCC
    mfcc_data = pd.DataFrame()

    for file_name in os.listdir(input_folder_path):
        if file_name.endswith('.wav'):
            file_path = os.path.join(input_folder_path, file_name)

            if feature_engineering_for_file(file_path, n_mfcc) is not None:
    
                mfcc_array, zcr, pitch = feature_engineering_for_file(file_path, n_mfcc)
                
                mfcc_df = pd.DataFrame(mfcc_array.reshape(1, -1), columns=[f'mfcc_{i+1}' for i in range(len(mfcc_array))])

                mfcc_df['label'] = label
                mfcc_df['file'] = file_name
                mfcc_df['ZCR'] = zcr
                mfcc_df['Pitch'] = pitch
                # mfcc_df['spectralFlux'] = compute_spectral_flux(spectrogram)

                mfcc_data = pd.concat([mfcc_data, mfcc_df], ignore_index=True)
        
    return mfcc_data

In [115]:
# mfcc_data = process_file('Angry', 'D:/data_analysis/speech_emotion_recognition/data/combined_data/Angry', 30)
# print(mfcc_data)

In [116]:
def process_folder(input_folder, n_mfcc, output_csv_path):
    # Tạo một DataFrame để lưu trữ dữ liệu MFCC
    mfcc_data = pd.DataFrame()

    # Duyệt qua tất cả các thư mục con trong thư mục đầu vào
    for root, dirs, files in os.walk(input_folder):
        for folder in dirs:
            # Xác định đường dẫn của thư mục con trong thư mục đầu vào
            subdirectory_input = os.path.join(root, folder)
            # Xử lý các tệp trong thư mục con và thêm dữ liệu MFCC vào DataFrame
            mfcc_df = process_file(folder, subdirectory_input, n_mfcc)
            mfcc_data = pd.concat([mfcc_data, mfcc_df], ignore_index=True)

    mfcc_data.to_csv(output_csv_path, index=False)
    print(f"Đã lưu dữ liệu MFCC vào {output_csv_path}")

In [117]:
process_folder(r'D:\data_analysis\speech_emotion_recognition\data\EnglishDataset\cleaned_data', 20, 'cleaned_data.csv')
# process_folder(r'D:\data_analysis\speech_emotion_recognition\data\VietnameseDataset', 40, 'test_data.csv')
# process_folder(r'D:\data_analysis\speech_emotion_recognition\audio_test\Bong_Dung_Muon_Khoc\divide_segment\1', 20, 'test_data.csv')

Đã lưu dữ liệu MFCC vào cleaned_data.csv


In [119]:
process_folder(r'D:\data_analysis\speech_emotion_recognition\data\EnglishDataset\normalized_data', 20, 'normalized_data.csv')

Đã lưu dữ liệu MFCC vào normalized_data.csv
